In [9]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    average_precision_score,
    precision_recall_curve
)
import os

os.makedirs("oof_predictions", exist_ok=True)

# Load OOF predictions
p_lstm = np.load("oof_predictions/oof_lstm.npy").ravel()
p_xgb = np.load("oof_predictions/xgb_oof_probs.npy").ravel()
p_tabnet = np.load("oof_predictions/oof_tabnet.npy").ravel()

# Load ONE target file (they should all be identical)
y = np.load("oof_predictions/y_oof_xgb.npy").astype(int).ravel()

assert len(p_lstm) == len(y)
assert len(p_xgb) == len(y)
assert len(p_tabnet) == len(y)

print("Samples:", len(y))
print("Positive rate:", round(y.mean(), 3))

# Stack base predictions
X_stack = np.column_stack([p_lstm, p_xgb, p_tabnet])

# Meta learner
meta = LogisticRegression(class_weight="balanced", max_iter=1000)
meta.fit(X_stack, y)

# Meta predictions
p_meta = meta.predict_proba(X_stack)[:, 1]

# Threshold tuning
precision, recall, thresholds = precision_recall_curve(y, p_meta)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_idx = np.argmax(f1_scores[:-1])
best_threshold = thresholds[best_idx]

y_pred_default = (p_meta >= 0.5).astype(int)
y_pred_tuned = (p_meta >= best_threshold).astype(int)

print("\nSTACKING RESULTS")
print("AUC:", round(roc_auc_score(y, p_meta), 3))
print("AUPRC:", round(average_precision_score(y, p_meta), 3))
print("F1 (0.5):", round(f1_score(y, y_pred_default, zero_division=0), 3))
print("F1 (tuned):", round(f1_score(y, y_pred_tuned, zero_division=0), 3))
print("Best threshold:", round(best_threshold, 3))

# ✅ SAVE STACKING OUTPUT
np.save("oof_predictions/oof_meta.npy", p_meta)
np.save("oof_predictions/y_oof_meta.npy", y)

print("\nSaved:")
print("oof_predictions/oof_meta.npy")
print("oof_predictions/y_oof_meta.npy")

Samples: 101766
Positive rate: 0.112

STACKING RESULTS
AUC: 0.682
AUPRC: 0.226
F1 (0.5): 0.274
F1 (tuned): 0.286
Best threshold: 0.571

Saved:
oof_predictions/oof_meta.npy
oof_predictions/y_oof_meta.npy
